# Steam API

In [21]:
import pandas as pd
import requests
# Get the api key
with open('steam_apikey.txt', 'r') as file:
    apikey = file.read()

base_url = 'http://api.steampowered.com/'
endpoint = 'IPlayerService/GetOwnedGames/v0001/?'
steamid = '76561198039160689'

url = base_url + endpoint + 'key=' + apikey + '&steamid=' + steamid + '&include_appinfo=true' + '&format=json'


In [22]:
r = requests.get(url)

r.json()

{'response': {'game_count': 301,
  'games': [{'appid': 20,
    'name': 'Team Fortress Classic',
    'playtime_forever': 0,
    'img_icon_url': '38ea7ebe3c1abbbbf4eabdbef174c41a972102b9',
    'playtime_windows_forever': 0,
    'playtime_mac_forever': 0,
    'playtime_linux_forever': 0,
    'playtime_deck_forever': 0,
    'rtime_last_played': 0,
    'content_descriptorids': [2, 5],
    'playtime_disconnected': 0},
   {'appid': 50,
    'name': 'Half-Life: Opposing Force',
    'playtime_forever': 0,
    'img_icon_url': '04e81206c10e12416908c72c5f22aad411b3aeef',
    'playtime_windows_forever': 0,
    'playtime_mac_forever': 0,
    'playtime_linux_forever': 0,
    'playtime_deck_forever': 0,
    'rtime_last_played': 0,
    'content_descriptorids': [2, 5],
    'playtime_disconnected': 0},
   {'appid': 70,
    'name': 'Half-Life',
    'playtime_forever': 0,
    'img_icon_url': '95be6d131fc61f145797317ca437c9765f24b41c',
    'playtime_windows_forever': 0,
    'playtime_mac_forever': 0,
    'pl

In [27]:
steam_df = pd.DataFrame(pd.DataFrame(r.json()).values[1][0])
steam_df.to_csv('steam_data.csv', index=False)

In [ ]:
steam_df

,appid,name,playtime_forever,img_icon_url,playtime_windows_forever,playtime_mac_forever,playtime_linux_forever,playtime_deck_forever,rtime_last_played,content_descriptorids,playtime_disconnected,has_community_visible_stats,playtime_2weeks,has_leaderboards
0,20,Team Fortress Classic,0,38ea7ebe3c1abbbbf4eabdbef174c41a972102b9,0,0,0,0,0,"[2, 5]",0,NaN,NaN,NaN
1,50,Half-Life: Opposing Force,0,04e81206c10e12416908c72c5f22aad411b3aeef,0,0,0,0,0,"[2, 5]",0,NaN,NaN,NaN
2,70,Half-Life,0,95be6d131fc61f145797317ca437c9765f24b41c,0,0,0,0,0,"[2, 5]",0,NaN,NaN,NaN
3,130,Half-Life: Blue Shift,0,b06fdee488b3220362c11704be4edad82abeed08,0,0,0,0,0,"[2, 5]",0,NaN,NaN,NaN
4,220,Half-Life 2,0,fcfb366051782b8ebf2aa297f3b746395858cb62,0,0,0,0,0,"[2, 5]",0,True,NaN,NaN
5,340,Half-Life 2: Lost Coast,0,795e85364189511f4990861b578084deef086cb1,0,0,0,0,0,"[2, 5]",0,NaN,NaN,NaN
6,280,Half-Life: Source,0,b4f572a6cc5a6a84ae84634c31414b9123d2f26b,0,0,0,0,0,NaN,0,NaN,NaN,NaN
7,360,Half-Life Deathmatch: Source,0,40b8a62efff5a9ab356e5c56f5c8b0532c8e1aa3,0,0,0,0,0,"[2, 5]",0,NaN,NaN,NaN
8,320,Half-Life 2: Deathmatch,0,795e85364189511f4990861b578084deef086cb1,0,0,0,0,0,"[2, 5]",0,NaN,NaN,NaN
9,380,Half-Life 2: Episode One,0,795e85364189511f4990861b578084deef086cb1,0,0,0,0,0,"[2, 5]",0,True,NaN,NaN
